In [ ]:
import torch
import gc
from unsloth import FastLanguageModel
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

In [ ]:
##Загрузка модели и эмбедов
# --- КОНФИГУРАЦИЯ ---
# Для 20GB VRAM: 
# Безопасно: "unsloth/Qwen2.5-14B-Instruct-bnb-4bit" (Контекст ~16k)
# Экстремально: "unsloth/Qwen2.5-32B-Instruct-bnb-4bit" (Контекст ~2-4k)
MODEL_ID = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit" 
MAX_SEQ_LEN = 8192 

# 1. Загрузка LLM на GPU
print(f"Загрузка LLM: {MODEL_ID}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

# Создаем пайплайн для LangChain
text_generation_pipeline = torch.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024, # Длина ответа
    temperature=0.1,     # Низкая температура для фактов
    repetition_penalty=1.15,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 2. Загрузка Эмбеддингов на CPU (!!! Важно для экономии памяти)
print("Загрузка модели эмбеддингов (CPU)...")
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={'device': 'cpu'}, # Вся работа идет в RAM, не VRAM
    encode_kwargs={'normalize_embeddings': True}
)
print("Система готова!")

In [ ]:
##Смешная векторная база с чанками. Может пригодиться
def create_db(file_paths):
    docs = []
    for file_path in file_paths:
        if file_path.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        else:
            loader = TextLoader(file_path, encoding="utf-8")
        docs.extend(loader.load())
    
    # Режем текст. chunk_size=800 — оптимально для захвата смысла.
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    splits = splitter.split_documents(docs)
    
    # Создаем базу. 
    # Если база уже есть, удаляем старую коллекцию, чтобы не было дублей
    vectorstore = Chroma.from_documents(
        documents=splits, 
        embedding=embeddings,
        collection_name="rag_collection"
    )
    return vectorstore.as_retriever(search_kwargs={"k": 3}) # k=3 фрагмента

# Пример использования (создадим файл для теста)
with open("tech_guide.txt", "w", encoding="utf-8") as f:
    f.write("Qwen2.5-32B — отличная модель, но требует много памяти. Для 20GB VRAM лучше использовать версию 14B. Unsloth ускоряет инференс в 2 раза.")

retriever = create_db(["tech_guide.txt"])
print("База знаний обновлена.")

In [ ]:
# Шаблон промпта. Qwen любит четкие инструкции "System" и "User".
template = """<|im_start|>system
Ты — профессиональный аналитик. Твоя задача — отвечать на вопросы, строго основываясь на предоставленном контексте.
Если информации в контексте нет, честно скажи "Я не нашел информации в документах".
Не придумывай факты. Отвечай на русском языке.

Контекст:
{context}<|im_end|>
<|im_start|>user
Вопрос: {question}<|im_end|>
<|im_start|>assistant
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Сборка цепочки: Поиск -> Форматирование -> Промпт -> LLM -> Парсинг ответа
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- ТЕСТ ---
query = "Какую модель лучше выбрать для 20GB VRAM?"
print(f"Вопрос: {query}\n")
response = rag_chain.invoke(query)
print(f"Ответ:\n{response}")

Часть 2: Гайд по Промпт-инжинирингу
Промпт — это интерфейс программирования LLM на естественном языке. Для Qwen (и других современных моделей) работает формула "CO-STAR" или ее упрощенные вариации.
1. Универсальная формула промпта (CO-STAR)
Используйте эту структуру для сложных задач (саммаризация, анализ, кодинг):
C (Context): Введи в курс дела.
O (Objective): Какова цель? Что нужно сделать?
S (Style): В каком стиле писать (деловой, код, для ребенка)?
T (Tone): Эмоциональная окраска (строгий, дружелюбный).
A (Audience): Для кого этот ответ?
R (Response): Формат ответа (JSON, список, Markdown).
Пример промпта (Саммаризация):
(Context) Я загрузил техническую документацию по серверному оборудованию.
(Objective) Мне нужно выделить основные риски эксплуатации.
(Style) Технический, сухой язык.
(Response) Сделай маркированный список из 5 пунктов на русском языке.
2. Техники для улучшения качества
A. Zero-Shot vs Few-Shot
Zero-Shot: Просто просишь. "Переведи это: ..."
Few-Shot (Примеры): Даешь примеры "Вход -> Выход". Это самый мощный способ заставить модель следовать формату.
code
Text
Задача: Преврати описание в JSON.
Пример 1:
Текст: "Иван купил яблоко." -> JSON: {"person": "Иван", "action": "buy", "object": "apple"}

Текст: "Мария продала машину." -> JSON




B. Chain of Thought (CoT) — "Думай шаг за шагом"
Если задача требует логики (математика, дедукция), добавьте в промпт фразу:
"Давай подумаем шаг за шагом" (Let's think step by step).
Для Qwen это работает магически. Это заставляет модель генерировать промежуточные рассуждения перед финальным ответом.



C. Ролевая модель (Persona)
Всегда задавайте роль. LLM — это актер.
Плохо: "Напиши код для сайта."
Хорошо: "Ты — Senior Python Backend разработчик с 10-летним стажем. Напиши безопасный код на FastAPI..."



3. Специфика Qwen (Qwen2.5)
Qwen обучался на огромном массиве данных и хорошо понимает спец. токены.
Язык: Если вы спрашиваете на русском, но модель срывается на английский (бывает в коде), добавьте в конце: "Ответ должен быть строго на русском языке" или "Translate output to Russian".
Системный промпт: Qwen очень чувствителен к началу диалога. В коде (как в пайплайне выше) всегда оборачивайте инструкции в теги <|im_start|>system ... <|im_end|>.
Запреты: Модели плохо понимают отрицания ("НЕ делай это"). Лучше писать позитивные инструкции ("Вместо этого делай то").
Плохо: "Не пиши длинные предложения."
Хорошо: "Пиши короткими, рублеными фразами."
Шпаргалка для ваших задач



Задача: RAG (Ответ по документу)
"Ты — эксперт по [тема документа]. Используй только представленный ниже контекст для ответа. Не используй свои внешние знания. Если контекста недостаточно, скажи об этом. Цитаты из текста приветствуются."



Задача: Саммаризация
"Прочитай текст ниже. Твоя задача — создать краткую выжимку (Executive Summary). Выдели 3 главные идеи и ключевые выводы. Игнорируй вводные слова и воду. Формат: заголовок + буллет-поинты."




Задача: Файнтюн (Подготовка данных)
"Перепиши этот грязный текст в формат вопрос-ответ. Вопрос должен быть от лица новичка, а ответ — от лица профессионала. Соблюдай структуру JSON."